In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from scipy.spatial.distance import cdist #for euclidean distance of consistency
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import uuid #to generate random id
import pickle #to save/load list of selected hens

#test equal variance
from scipy.stats import levene

#clustering
from sklearn.cluster import AgglomerativeClustering 

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#interpolate curves for clustering among birds with not exact same days tracked
#from scipy.interpolate import interp1d

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score, r2_score,\
mean_squared_error, mean_absolute_error, explained_variance_score#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
from scipy.stats import pearsonr, spearmanr 
import scipy.stats as stats

#PCA
from sklearn import decomposition

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns
import dexplot as dxp #for barplot
from mpl_toolkits.mplot3d import Axes3D

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, ts_visual, time_series_henColumn_tsRow, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
pal_class_treat = config.pal_class_treat
pal_treat = config.pal_treat
pal_class = config.pal_class
pal_interintre_treatment = config.pal_interintre_treatment
max_date_adaptability = config.max_date_adaptability
dico_pen_tr = config.dico_pen_tr
path_extracted_data_visual = os.path.join(path_extracted_data,'visual')
path_extracted_data_visual_corr = os.path.join(path_extracted_data,'visual','correlation')
path_extracted_data_visual_adap = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability')
print(id_run)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
rv = 0

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [5]:
#selected hens
li_selected_hens = pickle.load(open(os.path.join(path_extracted_data_visual_adap,'li_selected_hens.pkl'), 'rb'))
len(li_selected_hens)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


131

In [6]:
#focal birds info (one row per bird)
df_FB = pd.read_csv(os.path.join(path_extracted_data,id_run+'df_FOCALBIRDS.csv'), sep=';', parse_dates=['InitialStartDate'],
                     dayfirst=True) 
df_FB = df_FB[df_FB['HenID'].isin(li_selected_hens)]
df_FB['percentage_of_gain_weight'] = df_FB.apply(lambda x: (x['weight 23-11-2020']-x['29-09 weight'])/x['29-09 weight']*100, axis=1)
print(df_FB.shape)
df_FB.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(131, 12)


,HenID,PenID,CLASS,R-Pen,InitialStartDate,29-09 weight,10-12 juin weight,weight 23-11-2020,weight 04-01-2021,weight 01-02-21,Treatment,percentage_of_gain_weight
0,hen_1,9,EPI,1.0,2020-06-10,1134.0,57.0,1696.5,1787.8,1800.9,OFH,49.603175
1,hen_10,11,LEXP,1.0,2020-06-10,1070.0,70.4,1488.3,1628.4,1602.1,OFH,39.093458
3,hen_101,5,MEXP,3.0,2020-06-10,1082.0,66.0,1625.7,1751.1,1723.4,OFH,50.249538


In [7]:
#cleaned data of the tracking system movements
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', 
                 parse_dates=['Timestamp', 'date'], dayfirst=True) 
df = df[df['Timestamp']<=max_date_adaptability+dt.timedelta(days=2)] #add two days to have the next observations as well when doing the 1sec ts
df['hour'] = df['Timestamp'].map(lambda x: x.hour)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x-dt.timedelta(seconds=x.second)))
df = df[df['HenID'].isin(li_selected_hens)] 
df.drop('duration', axis=1, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(300191, 17)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open,hour,time
0,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:07:00,2020-09-29,2020-09-29 09:08:26.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:07:00
1,hen_7,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:08:12,2020-09-29,2020-09-29 09:12:16.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:08:00
2,hen_10,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:19:19,2020-09-29,2020-09-29 09:20:27.000,NaN,NaN,3_Zone,NaN,NaN,False,False,9,09:19:00


In [8]:
#daily variables (one row per (henID, date))
df_daily = pd.read_csv(os.path.join(path_extracted_data, 'daily_ALL_Variable_Tranformed.csv'), sep=';',
                     parse_dates=['level','FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone',
                              'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone', 
                              'Nestbox_time_of_first_staid_longer_than900sec',
                              'duration_last-firsttransition_mn'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by dates
df_daily = df_daily[df_daily['level']<=max_date_adaptability] 
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]

#filter the selected hens
df_daily = df_daily[df_daily['HenID'].isin(li_selected_hens)] 

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
print(df_daily.shape)
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)

#remove the days where the night had movement recorded but not the days
print(df_daily.shape)
#display(df_daily[df_daily.isna().any(axis=1)])
df_daily = df_daily[~df_daily['verification_daily_total_duration'].isnull()]
print(df_daily.shape)

df_daily['dur_values_normalized'].replace('[nan, nan, nan, nan, nan]','[np.nan,np.nan,np.nan,np.nan,np.nan]', inplace=True)
df_daily['dur_values_normalized'] = df_daily['dur_values_normalized'].map(lambda x: eval(x))
df_daily['duration_last-firsttransition_mn'] = df_daily['duration_last-firsttransition_mn'].astype(float)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(27214, 199)
(6721, 199)
(6721, 199)
(6721, 199)
(6382, 199)
(6382, 199)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,clustering_ALL,weeks_in_laying_barn
0,hen_1,2020-09-30,0.0,18886.0,3488.0,518.0,5908.0,28800.0,"[0.0, 18886.0, 3488.0, 518.0, 5908.0]","[0.0, 0.6557638888888889, 0.12111111111111111,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,hen_1,2020-10-01,0.0,9310.0,5636.0,0.0,17454.0,32400.0,"[0.0, 9310.0, 5636.0, 0.0, 17454.0]","[0.0, 0.2873456790123457, 0.17395061728395061,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,hen_1,2020-10-02,0.0,0.0,551.0,0.0,31849.0,32400.0,"[0.0, 0.0, 551.0, 0.0, 31849.0]","[0.0, 0.0, 0.017006172839506173, 0.0, 0.982993...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [9]:
#download the MLPs vectors
dico_pen_bin_zone_level_h = pickle.load(open(os.path.join(path_extracted_data_visual_adap, 
                                                 id_run+'dico_pen_bin_zone_level_h.pkl'), 'rb'))
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data_visual_adap, 
                                                     id_run+'dico_pen_bin_level_h.pkl'), 'rb'))
#dico of hen as keys and pen as values
df_daily['PenID'] = df_daily['PenID'].map(lambda x: 'pen'+(str(int(x))))
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Compute the distance across pairs of MLPS

In [10]:
#compute the CHI2DISTANCE & DTW for each bird with all other birds (because we need this exact same pair to do the visual)
random.seed(rv)
#for efficiency and testing select a subset of animals to be 50 hens, but later we dontn want to restrcit to the pen nor the
#number of animals
#nbr_hens = 70
#randomly pick more than one element from the list without repeating elements.
#li_selected_hens_rv = random.sample(li_selected_hens, nbr_hens)
li_selected_hens_rv = li_selected_hens
li_df = []
li_zone = list(dico_pen_bin_zone_level_h['pen3'][15].keys())
df.sort_values(['Timestamp'], inplace=True)
df_daily_h = df_daily[df_daily['DOA']<=119+30].copy()
li_date = set(df_daily_h['level'].tolist())
df_doa = df_daily[['level','DOA','WOA']].drop_duplicates()
df_doa.head(3)
df_daily_h = df_daily_h[df_daily_h['HenID'].isin(li_selected_hens_rv)].copy()

for d1 in tqdm.tqdm(sorted(list(li_date))):
    doa = df_doa[df_doa['level']==d1]['DOA'].values[0]
    woa = df_doa[df_doa['level']==d1]['WOA'].values[0]
    li_hen_d1 = df_daily_h[df_daily_h['level']==d1]['HenID'].unique()
    #its a symmetric measure, so we only do each combination once
    for i,h1 in enumerate(li_hen_d1):

        #for each hen we want to compute its difference with all other birds 
        for h2 in li_hen_d1[i+1:]:
            
            ############ Overall aviary usage similarity across days
            l1_chi2 = df_daily_h[(df_daily_h['HenID']==h1)&(df_daily_h['level']==d1)]['dur_values_normalized'].values[0]
            l2_chi2 = df_daily_h[(df_daily_h['HenID']==h2)&(df_daily_h['level']==d1)]['dur_values_normalized'].values[0]

            dico_ = {'level1':d1, 'level2':d1, 'henID1':h1, 'henID2':h2, 
                     'dur_values_normalized_henID1':l1_chi2,'dur_values_normalized_henID2':l2_chi2,
                     'chi2distance':chi2_distance(l1_chi2,l2_chi2),'DOA':doa,'WOA':woa}

            ############# Specifics' zones usage similarity across days
            for nbr_binmn in li_binmn:
                #ALL zone together with chi2distance
                l1_chi2 = dico_pen_bin_level_h[dico_h_p[h1]][nbr_binmn][d1][h1]
                l2_chi2 = dico_pen_bin_level_h[dico_h_p[h2]][nbr_binmn][d1][h2]
                dico_['nbr_obs_chi2all_'+str(nbr_binmn)] = len(l1_chi2)
                li_chi2 = [chi2_distance(l1_chi2[i],l2_chi2[i]) for i in range(0,len(l1_chi2))]
                dico_['li_chi2_'+str(nbr_binmn)] = li_chi2
                dico_['chi2distance_ALL_'+str(nbr_binmn)] = sum(li_chi2)
                
                #per zone with DTW
                for ZONE in li_zone:
                    dtw_value = np.nan
                    try:
                        l1_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h1]][nbr_binmn][ZONE][d1][h1], dtype=np.double)
                        l2_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h2]][nbr_binmn][ZONE][d1][h2], dtype=np.double)
                        #compute distance measure
                        #psi: #of start and end points of a sequence can be ignored if this would lead to a lower distance
                        dtw_value, paths = dtw.warping_paths(l1_dtw, l2_dtw, window=dico_window[nbr_binmn], psi=0, penalty=penalty)
                    except Exception as e:
                        print(e)
                        sys.exit()
                        pass #dont do anything
                    dico_['DTW_'+str(nbr_binmn)+'_'+ZONE] = round(dtw_value,1)  
                    dico_['nbr_obs_l1_'+str(nbr_binmn)+'_'+ZONE] = len(l1_dtw)
                    dico_['nbr_obs_l2_'+str(nbr_binmn)+'_'+ZONE] = len(l2_dtw)
            li_df.append(dico_)

#turn inter and intra distances into a dataframe                
df_interintra = pd.DataFrame(li_df)
#df_interintra = df_interintra[~df_interintra['chi2distance'].isnull()]
df_interintra = df_interintra.sort_values('DOA', ascending=True)
df_interintra['weeks_in_laying_barn'] = df_interintra['DOA'].map(lambda x: int((x-119)/7)+1)
df_interintra['hen_pair'] = df_interintra.apply(lambda x: '-'.join(sorted([x['henID1'],x['henID2']])), axis=1)
df_interintra['DTW_15_all'] = df_interintra[['DTW_15_1_Zone', 'DTW_15_2_Zone', 'DTW_15_3_Zone','DTW_15_4_Zone','DTW_15_5_Zone']].sum(axis=1)
li_col = list(df_interintra.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
df_interintra.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
df_interintra.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
print(df_interintra.shape)
display(df_interintra.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|███████████████████████████████████████████████████████████████████████████████| 29/29 [6:29:51<00:00, 806.59s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-a2c1bdca366b>", line 71, in <module>
    df_interintra.to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+str(rv)+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
  File "C:\Users\camil\anaconda3\lib\site-packages\pandas\core\generic.py", line 3167, in to_csv
    formatter.save()
  File "C:\Users\camil\anaconda3\lib\site-packages\pandas\io\formats\csvs.py", line 206, in save
    self._save()
  File "C:\Users\camil\anaconda3\lib\site-packages\pandas\io\formats\csvs.py", line 328, in _save
    self._save_chunk(start_i, end_i)
  File "C:\Users\camil\anaconda3\lib\site-packages\pandas\io\formats\csvs.py", line 362, in _save_chunk
    libwriters.write_csv_rows(self.data, ix, self.nlevels, self.cols, self.writer)
  File "pandas\_libs\writers.pyx", line 70,

TypeError: object of type 'NoneType' has no len()

In [11]:
li_col = list(df_interintra.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
df_interintra.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
df_interintra.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
print(df_interintra.shape)
display(df_interintra.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['li_chi2_5', 'li_chi2_10', 'li_chi2_15', 'li_chi2_20', 'li_chi2_30']
(212264, 102)


,level1,level2,henID1,henID2,dur_values_normalized_henID1,dur_values_normalized_henID2,chi2distance,DOA,WOA,nbr_obs_chi2all_5,...,nbr_obs_l2_30_5_Zone,DTW_30_4_Zone,nbr_obs_l1_30_4_Zone,nbr_obs_l2_30_4_Zone,DTW_30_1_Zone,nbr_obs_l1_30_1_Zone,nbr_obs_l2_30_1_Zone,weeks_in_laying_barn,hen_pair,DTW_15_all
0,2020-09-30,2020-09-30,hen_1,hen_10,"[0.0, 0.6557638888888889, 0.12111111111111111,...","[0.0, 0.40243055555555557, 0.5327083333333333,...",0.233889,119,17,288,...,48,21.0,48,48,0.0,48,48,1,hen_1-hen_10,315.1
5686,2020-09-30,2020-09-30,hen_2,hen_47,"[0.0, 0.3379861111111111, 0.23836805555555557,...","[0.0, 0.8774305555555556, 0.11180555555555556,...",0.341557,119,17,288,...,48,31.8,48,48,0.0,48,48,1,hen_2-hen_47,166.6
5685,2020-09-30,2020-09-30,hen_2,hen_46,"[0.0, 0.3379861111111111, 0.23836805555555557,...","[0.0, 0.2679861111111111, 0.059409722222222225...",0.110694,119,17,288,...,48,124.6,48,48,0.0,48,48,1,hen_2-hen_46,341.3


# Visualise the similarity between two individuals over time

In [ ]:
plt.hist(df_interintra['chi2distance']);